In [3]:
from torch.utils.data import Dataset, DataLoader
from utils.readers import VtkReader
import os
import numpy as np

In [4]:
main_directory = os.path.dirname(os.getcwd())
general_path =  os.path.join(main_directory, 'patients_data', '{}', 'train')
images_path = general_path.format('MRI_volumes', 'MRI')
masks_path = general_path.format('masks', 'mask')
print(images_path)
print(masks_path)

/home/jupyter/tfm/patients_data/MRI_volumes/train
/home/jupyter/tfm/patients_data/masks/train


In [6]:
patient_id = 8
patient_image_path = os.path.join(images_path, 'MRI_{}.vtk'.format(patient_id))
patient_mask_path = os.path.join(masks_path, 'mask_{}.vtk'.format(patient_id))
print(patient_image_path)
print(patient_mask_path)

/home/jupyter/tfm/patients_data/MRI_volumes/train/MRI_8.vtk
/home/jupyter/tfm/patients_data/masks/train/mask_8.vtk


In [7]:
image = VtkReader(patient_image_path)
mask = VtkReader(patient_mask_path)
print(image.shape)
print(image.shape)

(320, 320, 60)
(320, 320, 60)


In [43]:
class GeneratePatches(Dataset):
    def __init__(self, images_path, masks_path, num_patches, shuffle_images = False):
        self.images_path = images_path
        self.masks_path = masks_path
        self.num_patches = num_patches
        self.shuffle_images = shuffle_images
        
        self.images_files = self.get_files(self.images_path)
        self.masks_files = self.get_files(self.masks_path)
        
        assert len(self.images_files) == len(self.masks_files), 'Num.images is different from num. masks'
        
        self.patients_ids = self.get_patients_ids(self.images_files)
        self.num_patients = len(self.patients_ids)
    
    @staticmethod
    def get_patients_ids(files):
        return [int(file.split('/')[-1].split('.')[0].split('_')[1]) for file in files]
        
    @staticmethod
    def get_files(path):
        return [os.path.join(path, file) for file in os.listdir(path) if file.split('.')[-1] == 'vtk']  
    
    def __len__(self):
        return len(self.patients_ids) * self.num_patches
    
    def __getitem__(self, idx):
        patch_id, image_id = divmod(idx, self.num_patients) 
        print("image_id:"", image_id, "patch_id: ", patch_id)

SyntaxError: invalid syntax (<ipython-input-43-b386a554e117>, line 29)

In [41]:
ddd = GeneratePatches(images_path, masks_path, 8)
ddd[47]

5 7


In [ ]:
0 --> 0, 0
1 --> 0, 1
...
7 --> 0, 7
8 --> 1, 0

In [ ]:
6 pacientes
8 batches
0, 1, 2, 